In [ ]:
!pip install sqlalchemy
!pip install pandas
!pip install pyodbc
!pip install matplotlib
!pip install seaborn
!pip install kagglehub

from sqlalchemy import create_engine
import pandas as pd
import pyodbc
import matplotlib.pyplot as plt
import seaborn as sns
import kagglehub


In [ ]:
#Connect to the SQL Server
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=localhost;'
                      'Database=CustomerReviews1;'
                      'Trusted_Connection=yes;')

#Query to get data frame from the CustomerReviews1 table
query = "SELECT * FROM CustomerReviews2"
df = pd.read_sql(query, conn)
conn.close()

In [ ]:
# Download latest version of dataset from kaggle
path = kagglehub.dataset_download("chaudharyanshul/airline-reviews")
print("Path to dataset files:", path)

In [ ]:
#Read the csv file and make a data frame
full_path = path + '/BA_AirlineReviews.csv'
!mv $full_path .
df = pd.read_csv('./BA_AirlineReviews.csv')
df

In [ ]:
# Check for invalid date formats
invalid_dates = df[~df['Datetime'].str.match(r'^\d{4}-\d{2}-\d{2}$') | df['Datetime'].isnull()]
invalid_dates

In [ ]:
# Display the invalid dates
print("Invalid Dates:")
print(invalid_dates.count())

In [ ]:
#Explore the Data
# Renaming the 'ID' column
column_index_to_rename = 0 # Index of the 'ID' column
new_column_name = 'ID'

# Renaming the column using the 'rename' method
df.rename(columns={df.columns[column_index_to_rename]: new_column_name}, inplace=True)

print(df)                  # Displaying the DataFrame after renaming
print(df.info())           # Get info on data types and non-null counts
print(df.describe())       # Get basic statistics for numeric columns

In [ ]:
# Print the number of null values per column
print(df.isnull().sum())

In [ ]:
# Visualize the null values as a heatmap
plt.figure(figsize=(10, 6))
sns.heatmap(df.isnull(), cbar=False, cmap='viridis')
plt.show()

In [9]:
#Drop the specified columns
df.drop(columns =
    ['ID','ReviewHeader','ReviewBody','Aircraft','Wifi&Connectivity', 'InflightEntertainment' ], inplace=True)

In [ ]:
# Visualize the null values as a heatmap
plt.figure(figsize=(10, 6))
sns.heatmap(df.isnull(), cbar=False, cmap='viridis')
plt.show()

In [ ]:
# Step 2: 
# Identify object type columns
object_columns = df.select_dtypes(include='object').columns



for column in df.columns:
    # Fill the null values in numerical columns by the median of each SeatType category
    if df[column].dtypes == 'float64':
       df[column] = df.groupby('SeatType')[column].transform(lambda x: x.fillna(x.median()))
    #Drop rows with null values in object type columns
    elif df[column].dtype == 'object':
         df.dropna(subset=[column], inplace=True)


# Verify the result
print(df.info())  # Get info on data types and non-null counts
print(df.isnull().sum())  # Print the number of null values per column

In [ ]:
# Visualize the null values as a heatmap
plt.figure(figsize=(10, 6))
sns.heatmap(df.isnull(), cbar=False, cmap='viridis')
plt.show()

In [ ]:
# Visualize the Distribution
CatogiralColumns=['VerifiedReview','TypeOfTraveller','SeatType']
# Loop through each column in the DataFrame
for column in df.columns:
    # Check if the column is numeric
    if df[column].dtype == 'float64':
        #Plot histogram
        sns.histplot(df[column], bins=30, kde=True)
        plt.title(f'Distribution of {column}')
        plt.xlabel(column)
        plt.ylabel('Frequency')
        plt.show()
    elif df[column].dtype == 'bool' or column in CatogiralColumns:
        #Plot count plot for categorical columns
        plt.figure(figsize=(10, 4))
        sns.countplot(y=df[column])
        plt.title(f'Distribution of {column}')
        plt.ylabel(column)
        plt.xlabel('Count')
        plt.show()

In [15]:
df.to_csv('./CustomerService2.csv', index=False)

In [ ]:
#Database connection details
server = 'localhost'
database = 'CustomerReviews2'
driver = 'ODBC Driver 17 for SQL Server'

#Connection string to connect DB
connection_string = f"mssql+pyodbc://{server}/{database}?driver={driver}"
engine = create_engine(connection_string)

#Save the DataFrame to new DB and replace table if it exists
df.to_sql('CustomerService2', engine, if_exists='replace', index=False)
print("DataFrame saved successfully to SQL Server!")